In [1]:
import os
import contextlib

import pandas as pd
import seaborn as sns
from datetime import date
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import date, timedelta

In [2]:
def find_sheet_name(sheet_names):
    cpi_sheet_names = list(
        filter(lambda x: "tongmuc" in x.lower() and "22-23" not in x, sheet_names)
    )

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    cpi_sheet_names = list(filter(lambda x: "tongmuc" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    cpi_sheet_names = list(filter(lambda x: "tong muc" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    cpi_sheet_names = list(
        filter(lambda x: "tm" in x.lower() and "xnk" not in x.lower(), sheet_names)
    )

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    if len(sheet_names) == 43:
        return "23-24"

    cpi_sheet_names = list(filter(lambda x: "sheet1 (2)" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    raise RuntimeError(sheet_names, len(sheet_names))

In [3]:
en_to_vn = {
    "TOTAL": "TỔNG SỐ",
    "Retail sale": "Bán lẻ hàng hóa",
    "Accommodation and catering service": "Dịch vụ lưu trú, ăn uống",
    "Traveling service": "Du lịch lữ hành",
    "Other services": "Dịch vụ khác",
}

In [4]:
def get_tongmuc(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    df = df.dropna(axis=1, how="all")

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "Bán lẻ hàng hóa")
                | (df[df.columns[1]] == "Retail sale ")
                | (df[df.columns[1]] == "Thương nghiệp")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    df[df.columns[0]] = (
        df[df.columns[0]]
        .replace(r"\s+", " ", regex=True)
        .replace(r"\n", " ", regex=True)
    )

    df[df.columns[0]] = df[df.columns[0]].str.strip()

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "TỔNG SỐ")
                    | ((df[df.columns[0]] == "TOTAL"))
                    | ((df[df.columns[0]] == "Tổng số"))
                ].index[0]
            )
        )
    )

    df = df.reset_index(drop=True)

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "Dịch vụ khác")
                    | ((df[df.columns[0]] == "Other services"))
                    | ((df[df.columns[0]] == "Dịch vụ"))
                ].index[0]
                + 1,
                len(df.index),
            )
        )
    )

    df = df.dropna(axis=1, how="all")

    if len(df[df.columns[0]][df[df.columns[0]] == "TOTAL"].index) > 0:
        df[df.columns[0]] = df[df.columns[0]].map(en_to_vn)

    if len(df.columns) == 7:
        df = df.drop(columns=[df.columns[4]])
        df.columns = ["Phân Loại", "Pre-M", "M", "YTD", "M-YOY", "YTD-YOY"]
    elif len(df.columns) == 4 and int(date.split("-")[1]) == 1:
        df = df.drop(columns=[df.columns[2]])
        df.columns = ["Phân Loại", "M", "M-YOY"]
        df["YTD"] = df["M"]
        df["YTD-YOY"] = df["M-YOY"]
        df["Pre-M"] = np.nan
    elif len(df.columns) == 5 and int(date.split("-")[1]) == 1:
        df = df.drop(columns=[df.columns[3]])
        df.columns = ["Phân Loại", "Pre-M", "M", "M-YOY"]
        df["YTD"] = df["M"]
        df["YTD-YOY"] = df["M-YOY"]
    elif len(df.columns) == 5:
        df = df.drop(columns=[df.columns[3]])
        df.columns = ["Phân Loại", "M", "YTD", "YTD-YOY"]
        df["Pre-M"] = np.nan
        df["M-YOY"] = np.nan

    df = df[pd.to_numeric(df["M"], errors="coerce").notnull()]
    df = df[pd.to_numeric(df["YTD"], errors="coerce").notnull()]
    df = df[pd.to_numeric(df["YTD-YOY"], errors="coerce").notnull()]

    df["MOM"] = 100 * (df["M"] / df["Pre-M"]) - 100

    df["Date"] = date
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)

    df["Month"] = df["Date"].dt.strftime("%Y-%m")
    df["PreMonth"] = (
        df["Date"]
        .apply(
            lambda x: (
                x.replace(month=x.month - 1, day=1)
                if x.month > 1
                else x.replace(year=x.year - 1, month=12, day=1)
            )
        )
        .dt.strftime("%Y-%m")
    )

    # df["PreYearMonth"] = (
    #     df["Date"]
    #     .apply(lambda x: x.replace(year=x.year - 1, day=1))
    #     .dt.strftime("%Y-%m")
    # )

    return df


xl = pd.ExcelFile("../datas/2018-01-29-Bieu-01-2018-1.xlsx")

sheet_name = find_sheet_name(xl.sheet_names)

get_tongmuc(xl, sheet_name, "2018-01-29")

,Phân Loại,M,M-YOY,YTD,YTD-YOY,Pre-M,MOM,Date,Month,PreMonth
0,TỔNG SỐ,361073.369564,109.489143,361073.369564,109.489143,NaN,NaN,2018-01-29,2018-01,2017-12
1,Bán lẻ hàng hóa,272004.314564,108.281373,272004.314564,108.281373,NaN,NaN,2018-01-29,2018-01,2017-12
2,"Dịch vụ lưu trú, ăn uống",45089.807,114.702543,45089.807,114.702543,NaN,NaN,2018-01-29,2018-01,2017-12
3,Du lịch lữ hành,3931.896,139.271392,3931.896,139.271392,NaN,NaN,2018-01-29,2018-01,2017-12
4,Dịch vụ khác,40047.352,109.883508,40047.352,109.883508,NaN,NaN,2018-01-29,2018-01,2017-12


In [5]:
import os
import contextlib

dfs_tongmuc_origin = []
for file_name in sorted(
    os.listdir("../datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    if file_name.startswith(".~"):
        continue

    try:
        file_path = os.path.join("../datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2012:
            continue

        xl = pd.ExcelFile(file_path)

        df_tongmuc = get_tongmuc(
            xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}"
        )

        dfs_tongmuc_origin.append(df_tongmuc)
    except Exception as e:
        print(file_name)
        raise e

In [6]:
dfs_tongmuc = pd.concat(dfs_tongmuc_origin, ignore_index=True, axis=0)
dfs_tongmuc

,Phân Loại,Pre-M,M,YTD,M-YOY,YTD-YOY,MOM,Date,Month,PreMonth
0,TỔNG SỐ,529241.810992,535773.057468,4703401.100649,107.633866,108.766954,1.234076,2024-09-29,2024-09,2024-08
1,Bán lẻ hàng hóa,404976.899502,411272.176621,3630236.294753,107.701305,107.935716,1.554478,2024-09-29,2024-09,2024-08
2,"Dịch vụ lưu trú, ăn uống",62448.1,61727.36592,542962.435595,107.940005,113.567093,-1.154133,2024-09-29,2024-09,2024-08
3,Du lịch lữ hành,5256.777925,5058.37734,45606.55,101.86326,116.656543,-3.774186,2024-09-29,2024-09,2024-08
4,Dịch vụ khác,56559.974373,57715.137587,484595.827,107.36221,109.200397,2.042369,2024-09-29,2024-09,2024-08
...,...,...,...,...,...,...,...,...,...,...
943,Khu vực có vốn đầu tư nước ngoài,NaN,5617,5617,135.36,135.36,NaN,2012-01-30,2012-01,2011-12
944,Thương nghiệp,NaN,149653,149653,121.23,121.23,NaN,2012-01-30,2012-01,2011-12
945,"Khách sạn, nhà hàng",NaN,21991,21991,133.33,133.33,NaN,2012-01-30,2012-01,2011-12
946,Du lịch,NaN,1362,1362,111.1,111.1,NaN,2012-01-30,2012-01,2011-12


In [7]:
convert_name = {
    "TỔNG SỐ": "TỔNG SỐ",
    "Bán lẻ hàng hóa": "Bán lẻ hàng hóa",
    "Dịch vụ lưu trú, ăn uống": "Dịch vụ lưu trú, ăn uống",
    "Du lịch lữ hành": "Du lịch lữ hành",
    "Dịch vụ khác": "Dịch vụ khác",
    "Nhà nước": "Nhà nước",
    "Ngoài nhà nước": "Ngoài nhà nước",
    "Khu vực có vốn đầu tư nước ngoài": "Khu vực có vốn đầu tư nước ngoài",
    "Tổng số": "TỔNG SỐ",
    "Thương nghiệp": "Bán lẻ hàng hóa",
    "Khách sạn, nhà hàng": "Dịch vụ lưu trú, ăn uống",
    "Du lịch": "Du lịch lữ hành",
    "Dịch vụ": "Dịch vụ khác",
    "Tập thể": "Tập thể",
    "Tư nhân": "Tư nhân",
    "Cá thể": "Cá thể",
}

for pl in dfs_tongmuc["Phân Loại"].unique().tolist():
    assert pl in convert_name.keys(), dfs_tongmuc[dfs_tongmuc["Phân Loại"] == pl]

dfs_tongmuc["Phân Loại"] = dfs_tongmuc["Phân Loại"].map(convert_name)
dfs_tongmuc

,Phân Loại,Pre-M,M,YTD,M-YOY,YTD-YOY,MOM,Date,Month,PreMonth
0,TỔNG SỐ,529241.810992,535773.057468,4703401.100649,107.633866,108.766954,1.234076,2024-09-29,2024-09,2024-08
1,Bán lẻ hàng hóa,404976.899502,411272.176621,3630236.294753,107.701305,107.935716,1.554478,2024-09-29,2024-09,2024-08
2,"Dịch vụ lưu trú, ăn uống",62448.1,61727.36592,542962.435595,107.940005,113.567093,-1.154133,2024-09-29,2024-09,2024-08
3,Du lịch lữ hành,5256.777925,5058.37734,45606.55,101.86326,116.656543,-3.774186,2024-09-29,2024-09,2024-08
4,Dịch vụ khác,56559.974373,57715.137587,484595.827,107.36221,109.200397,2.042369,2024-09-29,2024-09,2024-08
...,...,...,...,...,...,...,...,...,...,...
943,Khu vực có vốn đầu tư nước ngoài,NaN,5617,5617,135.36,135.36,NaN,2012-01-30,2012-01,2011-12
944,Bán lẻ hàng hóa,NaN,149653,149653,121.23,121.23,NaN,2012-01-30,2012-01,2011-12
945,"Dịch vụ lưu trú, ăn uống",NaN,21991,21991,133.33,133.33,NaN,2012-01-30,2012-01,2011-12
946,Du lịch lữ hành,NaN,1362,1362,111.1,111.1,NaN,2012-01-30,2012-01,2011-12


In [8]:
merge_df = pd.merge(
    left=dfs_tongmuc,
    right=dfs_tongmuc,
    left_on=["Phân Loại", "Month"],
    right_on=["Phân Loại", "PreMonth"],
    how="left",
)
merge_df["Merged_M"] = np.where(
    merge_df["Pre-M_y"].isna(),
    merge_df["M_x"],
    merge_df["Pre-M_y"],
)
merge_df = merge_df.drop(
    columns=[
        "Pre-M_y",
        "M_y",
        "YTD_y",
        "M-YOY_y",
        "YTD-YOY_y",
        "MOM_y",
        "Date_y",
        "Month_y",
        "PreMonth_y",
    ]
)
merge_df = merge_df.rename(
    columns={
        "Pre-M_x": "Pre-M",
        "M_x": "E-M",
        "YTD_x": "YTD",
        "M-YOY_x": "M-YOY",
        "YTD-YOY_x": "YTD-YOY",
        "MOM_x": "E-MOM",
        "Date_x": "Date",
        "Month_x": "Month",
        "PreMonth_x": "PreMonth",
        "Merged_M": "M",
    }
)
merge_df["MOM"] = 100 * (merge_df["M"] / merge_df["Pre-M"]) - 100
merge_df[merge_df["Phân Loại"] == "TỔNG SỐ"]

,Phân Loại,Pre-M,E-M,YTD,M-YOY,YTD-YOY,E-MOM,Date,Month,PreMonth,M,MOM
0,TỔNG SỐ,529241.810992,535773.057468,4703401.100649,107.633866,108.766954,1.234076,2024-09-29,2024-09,2024-08,535773.057468,1.234076
5,TỔNG SỐ,524336.57269,526581.649614,4148354.881803,107.891149,108.458863,0.428175,2024-08-29,2024-08,2024-07,529241.810992,0.935513
10,TỔNG SỐ,521247.185803,528284.761717,3625721.421217,109.412339,108.676512,1.350142,2024-07-29,2024-07,2024-06,524336.57269,0.592691
15,TỔNG SỐ,516774.609545,522502.311721,3098691.785417,109.146817,108.595986,1.108356,2024-06-29,2024-06,2024-05,521247.185803,0.865479
20,TỔNG SỐ,520142.54587,519760.357083,2580175.221234,109.48295,108.652786,-0.073478,2024-05-29,2024-05,2024-04,516774.609545,-0.647503
...,...,...,...,...,...,...,...,...,...,...,...,...
898,TỔNG SỐ,NaN,191800,952157,NaN,120.811046,NaN,2012-05-29,2012-05,2012-04,191800,NaN
908,TỔNG SỐ,NaN,192432,762151,NaN,121.6,NaN,2012-04-30,2012-04,2012-03,192432,NaN
918,TỔNG SỐ,NaN,189056,569723,NaN,121.8,NaN,2012-03-29,2012-03,2012-02,189056,NaN
928,TỔNG SỐ,NaN,186463,380280,NaN,122.04,NaN,2012-02-29,2012-02,2012-01,186463,NaN


In [9]:
dfs_tongmuc = merge_df

In [10]:
dfs_tongmuc.to_csv("../csv/tongmuc.csv")

In [11]:
# dfs_tongmuc.loc[dfs_tongmuc["M-YOY"] < 30, "M-YOY"] = (
#     dfs_tongmuc.loc[dfs_tongmuc["M-YOY"] < 30, "M-YOY"] + 100
# )
# dfs_tongmuc.loc[dfs_tongmuc["YTD-YOY"] < 30, "YTD-YOY"] = (
#     dfs_tongmuc.loc[dfs_tongmuc["YTD-YOY"] < 30, "YTD-YOY"] + 100
# )

# dfs_tongmuc["M-YOY"] = dfs_tongmuc["M-YOY"] - 100
# dfs_tongmuc["YTD-YOY"] = dfs_tongmuc["YTD-YOY"] - 100

In [12]:
# nganh = "TỔNG SỐ"

# dfs_tongmuc_hanghoa = dfs_tongmuc[dfs_tongmuc["Phân Loại"] == nganh]
# dfs_tongmuc_hanghoa["PCT_YOY"] = dfs_tongmuc_hanghoa["M"].pct_change(-12)
# dfs_tongmuc["M-YOY-PCT"] = dfs_tongmuc_hanghoa["PCT_YOY"]

In [13]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_tongmuc[
    (
        (dfs_tongmuc["Phân Loại"] == "TỔNG SỐ")
        | (dfs_tongmuc["Phân Loại"] == "Bán lẻ hàng hóa")
        | (dfs_tongmuc["Phân Loại"] == "Dịch vụ lưu trú, ăn uống")
        # | (dfs_tongmuc["Phân Loại"] == "Du lịch lữ hành")
        # | (dfs_tongmuc["Phân Loại"] == "Dịch vụ khác")
    )
    # & (dfs_tongmuc["Date"] > "2021-01-01")
]

fig = px.area(df_show, x="Date", y="M", color="Phân Loại")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()